In [5]:
import pandas as pd
import os
import lightgbm as lgb
import xgboost as xgb
from sklearn.linear_model import SGDRegressor, LinearRegression, Ridge
from sklearn.preprocessing import MinMaxScaler
import math
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, log_loss
import matplotlib.pyplot as plt
import time
import warnings
warnings.filterwarnings('ignore')

import gc

In [10]:
path = 'C:/ZhangLI/Codes/DataSet/零基础入门金融风控-贷款违约预测'
train = pd.read_csv(path+'/train.csv')
# test = pd.read_csv(path+'/testA.csv')
train_new = train.drop(['id', 'issueDate', 'grade',
                      'subGrade','employmentLength', 'issueDate','earliesCreditLine'], axis=1)
train_new.fillna(0, inplace=True)
x_train_new = train_new.drop(['isDefault'], axis=1)
y_train_new = train['isDefault']
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_train_new, y_train_new, random_state=42)

In [8]:
train_new.isna().sum()

loanAmnt                  0
term                      0
interestRate              0
installment               0
employmentTitle           1
homeOwnership             0
annualIncome              0
verificationStatus        0
isDefault                 0
purpose                   0
postCode                  1
regionCode                0
dti                     239
delinquency_2years        0
ficoRangeLow              0
ficoRangeHigh             0
openAcc                   0
pubRec                    0
pubRecBankruptcies      405
revolBal                  0
revolUtil               531
totalAcc                  0
initialListStatus         0
applicationType           0
title                     1
policyCode                0
n0                    40270
n1                    40270
n2                    40270
n3                    40270
n4                    33239
n5                    40270
n6                    40270
n7                    40270
n8                    40271
n9                  

1. 回归任务中的加权平均法
pre = (pre1 + pre2 + pre3 +...+pren )/n
pre = 0.3pre1 + 0.3pre2 + 0.4pre3 
2. 分类算法中的投票
加权投票


In [11]:
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

clf1 = LogisticRegression(random_state=1)
clf2 = RandomForestClassifier(random_state=42)
clf3 = XGBClassifier(learning_rate=0.1, n_estimators=150, max_depth=4, min_child_weight=2, subsample=0.7,objective='binary:logistic')

clf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('xgb', clf3)])

train_new = train.drop(['id', 'issueDate', 'grade',
                      'subGrade','employmentLength', 'issueDate','earliesCreditLine'], axis=1)


clf.fit(x_train, y_train)
f1_score(y_test, clf.predict(x_test))

[16:38:16] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.09558482515536593

In [12]:
# 加权投票，这东西这么慢？
vclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('xgb', clf3)], voting='soft', weights=[2, 1, 1])
vclf.fit(x_train, y_train)
accuracy_score(y_test, vclf.predict(x_test))

[16:49:55] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.80124